In [95]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [96]:
import argparse
from Configs.Configs import BraninConfig
import Core.Utils as cf

In [97]:
import os
from Core.Dataset import BraninDataset, GenerateBraninDataset

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.nn.utils.weight_norm import weight_norm

from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [98]:
# genBranData = GenerateBraninDataset()
# filename = genBranData.GenerateData(100000, 'BraninTrainData')

In [99]:
taskName = "Branin"

# parser = argparse.ArgumentParser(description="DDPM parser")
# parser.add_argument("--validate", default=False, action="store_true")
# parser.add_argument("--runNumber", required=False, default=1)
# args = parser.parse_args()

config = BraninConfig()
config.runNumber = 1
config.trainDir = config.saveDir + f"/run{config.runNumber}/train/"

In [100]:
braninObj = cf.BraninDDPM(config)

In [101]:
print("Training Branin DDPM")
# os.makedirs(config.trainDir, exist_ok=False)

dataset = BraninDataset('Datasets/Branin/BraninTrainData_data.pth')
dataloader = DataLoader(dataset, batch_size=config.batchSize, shuffle=True,
                        num_workers=config.numWorkers)

ddpm = cf.DDPM(config).to(device)
optimizer = torch.optim.Adam(ddpm.parameters(), lr=config.learningRate,
                            weight_decay= config.weightDecay)
lrScheduler = torch.optim.lr_scheduler.LinearLR(optimizer, start_factor=1.0,
                                                end_factor=0.01, total_iters=config.nEpochs)

Training Branin DDPM


In [102]:
progressBar = tqdm(dataloader, leave=False)
lossEma = None
for trainData in progressBar:
#     print(f"Input shape: {trainData['x'].shape}, Label shape: {trainData['y'].shape}")
    optimizer.zero_grad()
    loss = ddpm(trainData["x"], trainData["y"])
    loss.backward()
    optimizer.step()
    break
#     if lossEma is None:
#         lossEma = loss.item()
#     else:
#         lossEma = 0.95*lossEma + 0.05*loss.item()
#     progressBar.set_description(f"EMA loss:{lossEma:.4f}")
# lrScheduler.step()

RuntimeError: The size of tensor a (64) must match the size of tensor b (2) at non-singleton dimension 1

In [91]:
from Core.Core import DiffusionModel, CommonNNFunctions, ResidualBlockSequences

In [92]:
x = trainData["x"]
y = trainData["y"]
noiseSched = DiffusionModel().GetLinearNoiseSchedule(*config.beta, config.nT)

batchSize = x.shape[0]
sampledTimes = torch.randint(1, config.nT + 1, (batchSize,)) # (B,)
stdNormalNoise = torch.randn_like(x) # (B, xDim)

# Forward Diffusion Model: x_t
xAtSampledTimes = (noiseSched["sqrtAlpha"][sampledTimes, None]*x +
                   noiseSched["sqrtOneMinusAlphaBar"][sampledTimes, None]*stdNormalNoise)

# For classifier free guidance
yBlock = torch.bernoulli(torch.zeros(batchSize) + config.dropProb)

tSteps = (sampledTimes/config.nT)[:, None]

In [93]:
class NNModel(nn.Module):
    def __init__(self, config) -> None:
        super().__init__()
        self.nnf = CommonNNFunctions()
        self.tEmbed = nn.Linear(1, config.nHiddenDims)
        self.yEmbed = nn.Linear(config.outputDim, config.nHiddenDims)
        
        self.tyEmbed = ResidualBlockSequences(nnf = self.nnf,
                                              inpDim = 2*config.nHiddenDims,
                                              outDim = config.nHiddenDims,
                                              nHidDims = config.nHiddenDims,
                                              nHidLyrs = 2,
                                              timeDim = config.nHiddenDims,
                                              actFnName = config.actFnName,
                                              actLyrName = config.actLayerName,
                                              wtNormInd = config.weightNormIndicator)
        
        self.denoise = ResidualBlockSequences(nnf = self.nnf,
                                              inpDim = config.inputDim,
                                              outDim = config.inputDim,
                                              nHidDims = config.nHiddenDims,
                                              nHidLyrs = config.nHiddenLayers,
                                              timeDim = config.nHiddenDims,
                                              actFnName = config.actFnName,
                                              actLyrName = config.actLayerName,
                                              wtNormInd = config.weightNormIndicator)
        
        print(self.tyEmbed)
    
    def forward(self, xInput, yInput, tSteps, yBlock):
        t = self.tEmbed(tSteps)
        y = self.yEmbed(yInput)
        y = y*(1 - yBlock) ## Classifier free guidance - classification embedding set to zero
        
        ty = torch.cat((t, y), dim=-1)
        
        print(ty.shape)
        ty = self.tyEmbed(ty, t)
        x = self.denoise(xInput, ty)
        return x
    
nnModel = NNModel(config)

ResidualBlockSequences(
  (inputBlock): Sequential(
    (0): Linear(in_features=128, out_features=64, bias=True)
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (resBlock): Sequential(
    (0): ResidualBlockWithTimeScaling(
      (block1): Linear(in_features=64, out_features=64, bias=True)
      (actLyr1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (actFn1): ReLU()
      (block2): Linear(in_features=64, out_features=64, bias=True)
      (actLyr2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (actFn2): ReLU()
      (tBlock): Sequential(
        (0): ReLU()
        (1): Linear(in_features=64, out_features=128, bias=True)
      )
    )
    (1): ResidualBlockWithTimeScaling(
      (block1): Linear(in_features=64, out_features=64, bias=True)
      (actLyr1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (ac

In [94]:
nnModel(xAtSampledTimes, y, tSteps, yBlock)

torch.Size([64, 128])


TypeError: forward() takes 2 positional arguments but 3 were given

In [ ]:
nnModel.tEmbed(tSteps)